In [1]:
import pandas as pd

In [32]:
conditions = pd.read_csv("C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\synthea\\output\\synthea500\\csv\\conditions.csv")

In [34]:
condition_codes = conditions.CODE

In [8]:
df_map = pd.read_csv("..\\ehr_preprocess\\helpers\\tls_Icd10cmHumanReadableMap_US1000124_20230301.tsv", sep="\t", )

In [35]:
# how many condition codes are in mapCategoryId in df_map?
df_map[df_map.mapCategoryId.isin(condition_codes)].shape

(0, 16)

In [42]:
condition_codes.nunique()

225

In [44]:
# what about correlationId
df_map[df_map.referencedComponentId.isin(condition_codes)].referencedComponentId.nunique()

217

In [49]:
len(df_map)

238278

In [52]:

# keep only the first
df_map = df_map.drop_duplicates(subset="referencedComponentId", keep="first")
snomed_to_icd10 = df_map.set_index("referencedComponentId").mapTarget.to_dict()

In [53]:
# map conditions.Code using referencedComponentId
conditions["CONCEPT"] = conditions.CODE.map(snomed_to_icd10)

In [58]:
conditions.CONCEPT.value_counts()
# strip away question mark at the end of CONCEPT if any
conditions.CONCEPT = conditions.CONCEPT.str.rstrip("?")
# remove . in the middle
conditions.CONCEPT = conditions.CONCEPT.str.replace(".", "")
# add D to the beginning of CONCEPT 
conditions.CONCEPT = "D" + conditions.CONCEPT
conditions = conditions.dropna(subset=["CONCEPT"])


In [59]:
conditions

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,CONCEPT
0,2013-09-16,2013-09-16,dabfdec7-929a-3269-fd84-ab7e6ecaa245,180dbc40-15f7-76aa-69b9-186a9d254418,314529007,Medication review due (situation),DDZ76.89
1,2014-09-22,2015-09-28,dabfdec7-929a-3269-fd84-ab7e6ecaa245,4b62c7f2-17e3-63f1-a435-59982d55e70e,314529007,Medication review due (situation),DDZ76.89
3,2015-08-02,2015-09-30,dabfdec7-929a-3269-fd84-ab7e6ecaa245,3c7523b4-954e-7d65-d6bb-192903bd04ed,110030002,Concussion injury of brain (disorder),DDS06.0X0
4,2015-08-02,2015-09-30,dabfdec7-929a-3269-fd84-ab7e6ecaa245,3c7523b4-954e-7d65-d6bb-192903bd04ed,62106007,Concussion with no loss of consciousness,DDS06.0X0
5,2016-10-03,2021-04-26,dabfdec7-929a-3269-fd84-ab7e6ecaa245,816b0a0d-2794-4ad1-1201-189672c0e023,314529007,Medication review due (situation),DDZ76.89
...,...,...,...,...,...,...,...
27972,2023-01-24,2023-03-21,a5aad816-cf2e-0a5e-1886-e249ef979ce5,81ab0780-33b9-37fd-48c7-8cb08da0a7ee,422650009,Social isolation (finding),DDT74.32X
27973,2023-03-21,2023-03-28,a5aad816-cf2e-0a5e-1886-e249ef979ce5,56cc358a-ca9f-a651-3d0e-31825ce4c574,314529007,Medication review due (situation),DDZ76.89
27975,2023-04-18,2023-04-18,a5aad816-cf2e-0a5e-1886-e249ef979ce5,e4226fd1-8052-07c9-e0b8-902537efed3a,314529007,Medication review due (situation),DDZ76.89
27977,2023-04-18,NaN,a5aad816-cf2e-0a5e-1886-e249ef979ce5,e4226fd1-8052-07c9-e0b8-902537efed3a,73595000,Stress (finding),DDZ73.3
